<a href="https://colab.research.google.com/github/yi-juho/-K_MHAS-_HuggingFace_dataset_MultiBERT_shared/blob/main/%5BK_MHAS%5D_HuggingFace_dataset_MultiBERT_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-MHaS (Korean Multi-label Hate Speech Dataset)

## Dataset loading

Loading the K-MHaS dataset from [HuggingFace](https://huggingface.co/datasets/jeanlee/kmhas_korean_hate_speech) and checking meta information (published @COLING2022)


In [1]:
!pip uninstall datasets fsspec -y
!pip install datasets==3.2.0 fsspec[http]==2024.9.0


Found existing installation: fsspec 2024.10.0
Uninstalling fsspec-2024.10.0:
  Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install transformers
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

kmhas_korean_hate_speech.py:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.24M [00:00<?, ?B/s]

default/validation/0000.parquet:   0%|          | 0.00/579k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})

In [5]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [6]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})

In [7]:
dataset.features

{'text': Value(dtype='string', id=None),
 'label': Sequence(feature=ClassLabel(names=['origin', 'physical', 'politics', 'profanity', 'age', 'gender', 'race', 'religion', 'not_hate_speech'], id=None), length=-1, id=None)}

In [8]:
# meta information

print(dataset.info.description)
print(dataset.info.homepage)

The K-MHaS (Korean Multi-label Hate Speech) dataset contains 109k utterances from Korean online news comments labeled with 8 fine-grained hate speech classes or Not Hate Speech class.
The fine-grained hate speech classes are politics, origin, physical, age, gender, religion, race, and profanity and these categories are selected in order to reflect the social and historical context.

https://github.com/adlnlp/K-MHaS


In [9]:
print(dataset.info.citation)

@inproceedings{lee-etal-2022-k,
    title = "K-{MH}a{S}: A Multi-label Hate Speech Detection Dataset in {K}orean Online News Comment",
    author = "Lee, Jean  and
      Lim, Taejun  and
      Lee, Heejun  and
      Jo, Bogeun  and
      Kim, Yangsok  and
      Yoon, Heegeun  and
      Han, Soyeon Caren",
    booktitle = "Proceedings of the 29th International Conference on Computational Linguistics",
    month = oct,
    year = "2022",
    address = "Gyeongju, Republic of Korea",
    publisher = "International Committee on Computational Linguistics",
    url = "https://aclanthology.org/2022.coling-1.311",
    pages = "3530--3538",
    abstract = "Online hate speech detection has become an important issue due to the growth of online content, but resources in languages other than English are extremely limited. We introduce K-MHaS, a new multi-label dataset for hate speech detection that effectively handles Korean language patterns. The dataset consists of 109k utterances from news commen

## Data preparation

- Prepare data from train, validation, and test dataset
- Multi-label is converted to multi-label one hot encodding

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [10]:
!pip install transformers
!pip install datasets

In [11]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

In [12]:
!pip install keras_preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00


In [13]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [14]:
# load train, validation, and test dataset from HuggingFace

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [15]:
# adding masking (able to remove this step depending on the model)

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train['text']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', validation['text']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test['text']))

In [16]:
# convert multi-label to multi-label binary (one hot encoding)
# [8] -> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

from sklearn.preprocessing import MultiLabelBinarizer

enc = MultiLabelBinarizer()

def multi_label(example):
    enc_label = enc.fit_transform(example['label'])
    float_arr = np.vstack(enc_label[:]).astype(float)
    update_label = float_arr.tolist()
    return update_label

train_labels = multi_label(train)
validation_labels = multi_label(validation)
test_labels = multi_label(test)

In [17]:
test_sentences[:5]

['[CLS] 그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터 [SEP]',
 '[CLS] "1. 8넘의 문재앙" [SEP]',
 '[CLS] "문재인 정권의 내로남불은 타의 추종을 불허하네. 자한당 욕할거리도 없음." [SEP]',
 '[CLS] "짱개들 지나간 곳은 폐허된다 ㅋㅋ" [SEP]',
 '[CLS] 곱창은 자갈치~~~~~ [SEP]']

In [18]:
test_labels[:5]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

## Prep for Pytorch

In [19]:
# Tokenizing : bert-base-multilingual-cased

tokenizer = BertTokenizer.from_pretrained('klue/bert-base', do_lower_case=False)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [20]:
MAX_LEN = 128

def data_to_tensor (sentences, labels):
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [21]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [22]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [23]:
print('testset size:', len(test_labels))
print('trainset size:', len(train_labels))
print('validset size:', len(validation_labels))

testset size: 21939
trainset size: 78977
validset size: 8776


## Multi-BERT model

### GPU setting

In [24]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [25]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### Model setting

In [26]:
num_labels = 9

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels, problem_type="multi_label_classification")
model.cuda()

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [27]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

# change epochs for improving results (our paper : epochs = 4)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [29]:
def multi_label_metrics(predictions, labels, threshold=0.5):

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true = labels
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    hamming = hamming_loss(y_true, y_pred)

    # return as dictionary
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc,
               'hamming_loss': hamming}

    return metrics

### Model training

In [30]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


500it [04:54,  1.69it/s]

  Batch   500  of  2,469.    Elapsed: 0:04:54.


1000it [09:49,  1.69it/s]

  Batch 1,000  of  2,469.    Elapsed: 0:09:50.


...

  Batch 1,500  of  2,469.    Elapsed: 0:14:47.


2000it [19:42,  1.69it/s]

  Batch 2,000  of  2,469.    Elapsed: 0:19:42.


2469it [24:18,  1.69it/s]


  Average training loss: 0.0492
  Training epcoh took: 0:24:19

Training complete!


In [31]:
# ========================================
#               Validation
# ========================================

print("")
print("Running Validation...")

t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))
print("Validation took: {:}".format(format_time(time.time() - t0)))


Running Validation...
Accuracy: 0.8113
F1 (Macro) Score: 0.7633
F1 (Micro) Score: 0.8561
F1 (Weighted) Score: 0.8552
ROC-AUC: 0.9160
Hamming Loss: 0.0360
Validation took: 0:00:55


### Evaluation

In [ ]:
# model save

# torch.save(model.state_dict(), path+"BERT_model.pt")

In [35]:
# load the saved model

path = '/content/model/'
torch.save(model.state_dict(), path+"BERT_multilabel_model.pt")
model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))
model.to(device)

<ipython-input-35-76ce191dc6fe>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [34]:
%mkdir model

In [36]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []
for batch in validation_dataloader:
 batch = tuple(t.to(device) for t in batch)
 b_input_ids, b_input_mask, b_labels = batch
 with torch.no_grad():
  outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
 logits = outputs[0]
 logits = logits.detach().cpu().numpy()
 label_ids = b_labels.to('cpu').numpy()
 for b in logits:
  accum_logits.append(list(b))
 for b in label_ids:
  accum_label_ids.append(list(b))
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

Accuracy: 0.8113
F1 (Macro) Score: 0.7633
F1 (Micro) Score: 0.8561
F1 (Weighted) Score: 0.8552
ROC-AUC: 0.9160


In [37]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("")
print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))
print("Test took: {:}".format(format_time(time.time() - t0)))

accum_results = []
accum_results.append(list(results.values()))

101it [00:19,  5.17it/s]

  Batch   100  of    686.    Elapsed: 0:00:20.


201it [00:39,  5.05it/s]

  Batch   200  of    686.    Elapsed: 0:00:39.


301it [00:58,  5.06it/s]

...

  Batch   500  of    686.    Elapsed: 0:01:38.


601it [01:58,  5.11it/s]

  Batch   600  of    686.    Elapsed: 0:01:58.


686it [02:15,  5.08it/s]



Accuracy: 0.8056
F1 (Macro) Score: 0.7730
F1 (Micro) Score: 0.8536
F1 (Weighted) Score: 0.8533
ROC-AUC: 0.9147
Hamming Loss: 0.0370
Test took: 0:02:15


### Break down evaluation

In [38]:
for i in range(num_labels):
    ith_label_ids, ith_logits = [], []

    for j, labels in enumerate(accum_label_ids):
        if len(np.where(labels)[0]) == i+1:
            ith_label_ids.append(accum_label_ids[j].tolist())
            ith_logits.append(accum_logits[j].tolist())

    ith_label_ids = np.array(ith_label_ids)
    ith_logits = np.array(ith_logits)

    if len(ith_label_ids) == 0 and len(ith_logits) == 0:
        continue

    results = multi_label_metrics(ith_logits, ith_label_ids)
    accum_results.append(list(results.values()))

    print('# of labels:', i+1)
    print("Accuracy: {0:.4f}".format(results['accuracy']))
    print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
    print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
    print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
    print("ROC-AUC: {0:.4f}".format(results['roc_auc']))
    print("Hamming Loss: {0:.4f}".format(results['hamming_loss']))

    print('\n')

# of labels: 1
Accuracy: 0.8317
F1 (Macro) Score: 0.7525
F1 (Micro) Score: 0.8523
F1 (Weighted) Score: 0.8546
ROC-AUC: 0.9227
Hamming Loss: 0.0333


# of labels: 2
Accuracy: 0.6568
F1 (Macro) Score: 0.7206
F1 (Micro) Score: 0.8662
F1 (Weighted) Score: 0.8749
ROC-AUC: 0.8976
Hamming Loss: 0.0559


# of labels: 3
Accuracy: 0.3759
F1 (Macro) Score: 0.6291
F1 (Micro) Score: 0.8159
F1 (Weighted) Score: 0.8143
ROC-AUC: 0.8486
Hamming Loss: 0.1080


# of labels: 4
Accuracy: 0.2400
F1 (Macro) Score: 0.5296
F1 (Micro) Score: 0.7886
F1 (Weighted) Score: 0.7762
ROC-AUC: 0.8210
Hamming Loss: 0.1644




In [39]:
from transformers import pipeline
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer,
device=0, max_length=512,
 return_all_scores=True, function_to_apply='sigmoid')
result = pipe('틀니들은 왜 그렇게 민폐를 끼치냐?')
print(result)
label_dict = {'LABEL_0' : '출신차별', 'LABEL_1' : '외모차별', 'LABEL_2' : '정치성향차별', 'LABEL_3': '혐오욕설', 'LABEL_4': '연령차별', 'LABEL_5': '성차별', 'LABEL_6' : '인종차별', 'LABEL_7': '종교차별', 'LABEL_8': '해당사항없음'}
def prediction(text):
 result = pipe(text)
 return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]
prediction('틀니들은 왜 그렇게 민폐를 끼치냐?')

Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.005725260823965073}, {'label': 'LABEL_1', 'score': 0.004730418790131807}, {'label': 'LABEL_2', 'score': 0.006375799421221018}, {'label': 'LABEL_3', 'score': 0.008912630379199982}, {'label': 'LABEL_4', 'score': 0.9810022115707397}, {'label': 'LABEL_5', 'score': 0.004596341401338577}, {'label': 'LABEL_6', 'score': 0.0014180458383634686}, {'label': 'LABEL_7', 'score': 0.002617949852719903}, {'label': 'LABEL_8', 'score': 0.008827072568237782}]]


['연령차별']

In [40]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.nn.functional import sigmoid, softmax
from itertools import chain
import itertools
import pickle


label_names = ['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
threshold = 0.1 # 임계값 설정

def predict_labels(text, model, tokenizer, label_names, threshold=0.1):
    # 텍스트를 모델의 입력 형식으로 인코딩
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 10,
                        padding = 'max_length',
                        truncation=True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                        )

    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    # 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)

    # 예측 결과에서 확률 추출
    logits = outputs.logits
    #print(logits)
    probs = sigmoid(logits)
    #print(probs)

    # CPU로 이동 후 numpy 배열로 변환
    probs = probs.detach().cpu().numpy()
    #print(probs)

    # 예측된 레이블 결정
    predicted_labels = [label_names[i] for i in range(len(label_names)) if probs[0][i] >= threshold]

    return predicted_labels

In [ ]:
text = "늙은 천주교 신자들은 다 속물이다"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} & Predicted labels: {predicted_labels}")

Text: 늙은 천주교 신자들은 다 속물이다 & Predicted labels: ['연령차별', '종교차별']


In [ ]:
label_names = ['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
threshold = 0.1
text = "못생긴 경상도 여자들은 나가라"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} -> Predicted labels: {predicted_labels}")

Text: 못생긴 경상도 여자들은 나가라 -> Predicted labels: ['출신차별', '외모차별', '성차별']


In [97]:
# VSCode 쪽 코드 (receive_data.py)
outputs_2 = []

import pickle
with open("/./content/model/attention_scores.pkl", "rb") as f:
    outputs_2 = pickle.load(f)

print(outputs_2)

[[0.00827506]
 [0.00343384]
 [0.9870287 ]
 [0.00724543]
 [0.00340026]
 [0.00394029]
 [0.00123554]
 [0.00331021]
 [0.0049621 ]]


In [41]:
result = pipe('문재앙')
print(result)

[[{'label': 'LABEL_0', 'score': 0.008275064639747143}, {'label': 'LABEL_1', 'score': 0.0034338391851633787}, {'label': 'LABEL_2', 'score': 0.9870287179946899}, {'label': 'LABEL_3', 'score': 0.007245426066219807}, {'label': 'LABEL_4', 'score': 0.0034002556931227446}, {'label': 'LABEL_5', 'score': 0.003940285183489323}, {'label': 'LABEL_6', 'score': 0.001235544914379716}, {'label': 'LABEL_7', 'score': 0.0033102051820605993}, {'label': 'LABEL_8', 'score': 0.004962099250406027}]]


In [ ]:
from transformers import pipeline

# GPU 사용 여부 확인
device = "cpu"

# 모델을 CPU로 이동
model = model.to(device)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device, max_length=512, return_all_scores=True)

result = pipe('깜둥이들은 왜 그렇게 민폐를 끼치냐?')
print(result)

Device set to use cpu


[[{'label': 'LABEL_0', 'score': 0.06834093481302261}, {'label': 'LABEL_1', 'score': 0.17408297955989838}, {'label': 'LABEL_2', 'score': 0.030496349558234215}, {'label': 'LABEL_3', 'score': 0.010288281366229057}, {'label': 'LABEL_4', 'score': 0.029566748067736626}, {'label': 'LABEL_5', 'score': 0.12651988863945007}, {'label': 'LABEL_6', 'score': 0.8385200500488281}, {'label': 'LABEL_7', 'score': 0.07148561626672745}, {'label': 'LABEL_8', 'score': 0.07514869421720505}]]


In [ ]:
text = "ㅅ발 천주교도들은 너무 말이 많아"
predicted_labels = predict_labels(text, model, tokenizer, label_names)
print(f"Text: {text} & Predicted labels: {predicted_labels}")

Text: ㅅ발 천주교도들은 너무 말이 많아 & Predicted labels: ['혐오욕설', '종교차별']


In [42]:
# 파일을 열고 읽은 다음 내용을 출력하는 코드
file_path = '/home/jyhan/HW-output-files/example.txt'  # 읽고자 하는 파일의 경로

try:
    with open(file_path, 'r') as file:  # 파일을 읽기 모드로 열기
        content = file.read()  # 파일의 내용을 전체 읽기
    print(content)  # 읽은 내용 출력
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")  # 파일이 존재하지 않을 경우 에러 메시지 출력

The file /home/jyhan/HW-output-files/example.txt does not exist.


In [43]:
import pickle

def pickle_to_text(pickle_file_path, text_file_path):
    # 피클 파일 불러오기
    with open(pickle_file_path, "rb") as f:
        data = pickle.load(f)

    # 텍스트 파일로 저장하기
    with open(text_file_path, "w") as f:
        for sublist in data:
            # 각 서브리스트를 반복하고 숫자를 문자열로 변환하여 저장
            for number in sublist:
                f.write(f"{number} ")
            f.write("\n\n\n")  # 각 서브리스트 끝에 줄바꿈 추가

In [44]:
import torch
import torch.nn.functional as F

def tanh_new(x) :
    result = (F.tanh(x) + 1) /2
    return result

In [45]:
def encode_number(num):
    """숫자를 7비트 2의 보수 형식으로 인코딩하는 함수, 음수 소수 부분을 고려"""
    num = float(num)  # 입력된 숫자가 문자열이라면, 실수형으로 변환

    # Extract integer and fractional parts
    int_num = int(num)
    frac_num = abs(num - int_num)

    # Round integer part towards zero if num is negative and has a fractional part
    if num < 0:
        if frac_num > 0:
            int_num = int_num - 1  # Round integer part one more negative
            frac_num = 1 - frac_num  # Subtract fractional part from 1 to make it positive

    # Clamp the values to fit within the 7-bit range
    if int_num < -64:
        int_num = -64
    elif int_num > 63:
        int_num = 63

    # Apply 2's complement if the number is negative
    if int_num < 0:
        int_num = (1 << 7) + int_num  # 1 << 7 is 128, representing the range of 7-bit integers

    # Format the number into 7-bit binary
    int_part_bin = format(int_num & 0b1111111, '07b')  # Only the last 7 bits are used

    # Convert the fractional part to binary (13 bits)
    decimal_part_bin = format(int(frac_num * (1 << 13)), '013b')

    # Combine all parts
    encoded = int_part_bin + '_' + decimal_part_bin
    return encoded

In [46]:
def encode_numbers_input(numbers):
    """리스트의 숫자들을 인코딩하는 함수"""
    return [encode_number(num) for num in numbers]

In [47]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.nn.functional import sigmoid, softmax
from itertools import chain
import itertools

In [48]:
def encode_number_output(num):
    """숫자를 3비트 정수와 13비트 소수 형식으로 인코딩하는 함수, 값은 0부터 1 사이의 양수"""
    num = float(num)  # 입력된 숫자가 문자열이라면, 실수형으로 변환

    # Clamp the values to fit within the 0 to 1 range
    if num < 0:
        num = 0
    elif num > 1:
        num = 1

    # Extract integer and fractional parts
    int_num = int(num)
    frac_num = num - int_num

    # Convert the fractional part to binary (13 bits)
    decimal_part_bin = format(int(frac_num * (1 << 13)), '013b')

    # Combine all parts
    encoded = '000' + '_' + decimal_part_bin
    return encoded

In [49]:
def encode_numbers_output(numbers):
    """리스트의 숫자들을 인코딩하는 함수"""
    return [encode_number_output(num) for num in numbers]

In [50]:
probs = [-3.0061, -4.5561, -3.7781, -4.6988, -1.8251, -4.0483, -4.8075,  4.6085, -5.1693]
encoded_numbers = encode_numbers_input(probs)
print(encoded_numbers)

probs = [0.0472, 0.0104, 0.0224, 0.0090, 0.1388, 0.0172, 0.0081, 0.9901, 0.0057]
encoded_numbers = encode_numbers_output(probs)
print(encoded_numbers)

['1111100_1111111001110', '1111011_0111000110100', '1111100_0011100011001', '1111011_0100110100011', '1111110_0010110011000', '1111011_1111001110100', '1111011_0011000101000', '0000100_1001101111000', '1111010_1101010010101']
['000_0000110000010', '000_0000001010101', '000_0000010110111', '000_0000001001001', '000_0010001110001', '000_0000010001100', '000_0000001000010', '000_1111110101110', '000_0000000101110']


In [51]:
probs = [0.0301, 0.0112, 0.0040, 0.7632, 0.0040, 0.0072, 0.0068, 0.8885, 0.0117]
encoded_numbers = encode_numbers_output(probs)
print(encoded_numbers)

['000_0000011110110', '000_0000001011011', '000_0000000100000', '000_1100001101100', '000_0000000100000', '000_0000000111010', '000_0000000110111', '000_1110001101110', '000_0000001011111']


In [52]:
def decode_number(encoded):
    """7비트 2의 보수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin, decimal_part_bin = encoded.split('.')

    # Integer part processing
    if int(int_part_bin, 2) & (1 << 6):  # Negative number (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 7)
    else:  # Positive number
        int_num = int(int_part_bin, 2)

    # Fractional part processing
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # Adjust for negative numbers
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# Test the function with provided values
encoded_values = [
    "0000000.0101010000101",
    "0000000.0010111100000",
    "0000000.0010110110000",
    "0000000.0010001111011",
    "0000000.0010001101110",
    "0000000.0001110010000",
    "0000000.0001100000110",
    "0000000.0000000000010",
    "1111111.1101100010111",
    "1111111.1101011100011"
]

decoded_values = [decode_number(value) for value in encoded_values]
decoded_values

[0.3287353515625,
 0.18359375,
 0.177734375,
 0.1400146484375,
 0.138427734375,
 0.111328125,
 0.094482421875,
 0.000244140625,
 -0.1534423828125,
 -0.1597900390625]

In [53]:
def decode_number(encoded):
    """7비트 2의 보수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    # 입력 문자열을 반으로 나누어 정수 부분과 소수 부분으로 분리
    mid_index = len(encoded) // 2
    int_part_bin = encoded[:mid_index]
    decimal_part_bin = encoded[mid_index:]

    # 정수 부분 처리
    if int(int_part_bin, 2) & (1 << 6):  # 음수인 경우 (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 7)
    else:  # 양수인 경우
        int_num = int(int_part_bin, 2)

    # 소수 부분 처리
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 음수 조정
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "00000000101010000101",
    "00000000010111100000",
    "00000000010110110000",
    "00000000010001111011",
    "00000000010001101110",
    "00000000001110010000",
    "00000000001100000110",
    "00000000000000000010",
    "11111111101100010111",
    "11111111101011100011"
]

decoded_values = [decode_number(value) for value in encoded_values]
decoded_values

[2.0787353515625,
 1.05859375,
 1.052734375,
 1.0150146484375,
 1.013427734375,
 0.111328125,
 0.094482421875,
 0.000244140625,
 894.0965576171875,
 894.0902099609375]

In [54]:
def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    if int(int_part_bin, 2) & (1 << 2):  # 음수인 경우 (2의 보수)
        int_num = int(int_part_bin, 2) - (1 << 3)
    else:  # 양수인 경우
        int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 음수 조정
    if int_num < 0:
        if frac_num > 0:
            int_num += 1
            frac_num = 1 - frac_num
        result = int_num - frac_num
    else:
        result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "0001000101001100",
    "0000011001010100",
    "0000000111011100",
    "0000000111010100",
    "0000000110110100",
    "0000000100000100",
    "0000000011010100",
    "0000000001100100",
    "0000000001010100",
    "0000000001000100"
]

decoded_values = [decode_int3_float13(value) for value in encoded_values]

# 결과 출력
for encoded, decoded in zip(encoded_values, decoded_values):
    print(f"Encoded: {encoded} -> Decoded: {decoded}")

Encoded: 0001000101001100 -> Decoded: 0.54052734375
Encoded: 0000011001010100 -> Decoded: 0.19775390625
Encoded: 0000000111011100 -> Decoded: 0.05810546875
Encoded: 0000000111010100 -> Decoded: 0.05712890625
Encoded: 0000000110110100 -> Decoded: 0.05322265625
Encoded: 0000000100000100 -> Decoded: 0.03173828125
Encoded: 0000000011010100 -> Decoded: 0.02587890625
Encoded: 0000000001100100 -> Decoded: 0.01220703125
Encoded: 0000000001010100 -> Decoded: 0.01025390625
Encoded: 0000000001000100 -> Decoded: 0.00830078125


In [55]:
def format_hex_input(hex_input):
    # 공백을 기준으로 입력값을 분할하여 리스트로 변환
    hex_list = hex_input.split()

    # 리스트를 문자열 형식으로 변환
    formatted_hex_list = "[" + ", ".join(f'"{hex_value}"' for hex_value in hex_list) + "]"

    return formatted_hex_list

# 입력값
hex_input = "06dc 05c4 0494 03c4 039c 02e4 017c 0144 00e4 00ac"
formatted_hex_list = format_hex_input(hex_input)

print(formatted_hex_list)

["06dc", "05c4", "0494", "03c4", "039c", "02e4", "017c", "0144", "00e4", "00ac"]


In [56]:
def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

def hexa_to_float(hex_strings):
    binary_strings = hexa_to_binary(hex_strings)
    float_values = [decode_int3_float13(binary_string) for binary_string in binary_strings]
    return float_values

# 새로운 입력값
hex_strings = ["06dc", "05c4", "0494", "03c4", "039c", "02e4", "017c", "0144", "00e4", "00ac"]

float_values = hexa_to_float(hex_strings)

# 변환된 결과 출력
for hex_string, float_value in zip(hex_strings, float_values):
    print(f"{float_value}")

0.21435546875
0.18017578125
0.14306640625
0.11767578125
0.11279296875
0.09033203125
0.04638671875
0.03955078125
0.02783203125
0.02099609375


In [57]:
def format_hex_input(hex_input):
    # 공백을 기준으로 입력값을 분할하여 리스트로 변환
    hex_list = hex_input.split()
    return hex_list

def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

def hexa_to_float(hex_strings):
    binary_strings = hexa_to_binary(hex_strings)
    float_values = [decode_int3_float13(binary_string) for binary_string in binary_strings]
    return float_values

def convert_hex_input_to_float(hex_input):
    hex_strings = format_hex_input(hex_input)
    float_values = hexa_to_float(hex_strings)
    return float_values

def chunk_floats(float_values, chunk_size=10):
    # float 값을 chunk_size 크기로 분할하고 각 chunk 사이에 두 개의 엔터를 추가
    chunked_result = ""
    for i in range(0, len(float_values), chunk_size):
        chunk = float_values[i:i + chunk_size]
        chunked_result += "\n".join(map(str, chunk)) + "\n\n"
    return chunked_result.strip()

# 입력값
hex_input = """
1404
026c
0234
0214
0184
017c
015c
001c
001c
0004
"""

float_values = convert_hex_input_to_float(hex_input)

# 변환된 결과 출력
chunked_result = chunk_floats(float_values)

print(chunked_result)

0.62548828125
0.07568359375
0.06884765625
0.06494140625
0.04736328125
0.04638671875
0.04248046875
0.00341796875
0.00341796875
0.00048828125


In [58]:
def decode_int3_float13(encoded):
    """3비트 정수 + 13비트 소수 형식으로 인코딩된 값을 float으로 디코딩하는 함수"""
    int_part_bin = encoded[:3]
    decimal_part_bin = encoded[3:]

    # 정수 부분 처리 (3비트)
    int_num = int(int_part_bin, 2)

    # 소수 부분 처리 (13비트)
    frac_num = int(decimal_part_bin, 2) / (1 << 13)

    # 결과 계산
    result = int_num + frac_num

    return result

# 제공된 값으로 함수를 테스트합니다
encoded_values = [
    "0000110101000100",
    "0000100000100100",
    "0000001101111100",
    "0000001011111100",
    "0000000111000100",
    "0000000100000100",
    "0000000001110100",
    "0000000001000100",
    "0000000000111100",
    "0000000000110100"
]

decoded_values = [decode_int3_float13(value) for value in encoded_values]

# 결과 출력
for decoded in decoded_values:
    print(decoded)


0.41455078125
0.25439453125
0.10888671875
0.09326171875
0.05517578125
0.03173828125
0.01416015625
0.00830078125
0.00732421875
0.00634765625


In [59]:
# 새로운 입력값
new_hex_strings = ["0d44", "0824", "037c", "02fc", "01c4", "0104", "0074", "0044", "003c", "0034"]
new_binary_strings = hexa_to_binary(new_hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(new_hex_strings, new_binary_strings):
    print(f"{binary_string}")


0000110101000100
0000100000100100
0000001101111100
0000001011111100
0000000111000100
0000000100000100
0000000001110100
0000000001000100
0000000000111100
0000000000110100


In [60]:
def hexa_to_binary(hex_strings):
    binary_strings = [bin(int(hex_string, 16))[2:].zfill(16) for hex_string in hex_strings]
    return binary_strings

# 새로운 입력값
hex_strings = ["069c", "046c", "045c", "0434", "02fc", "02a4", "026c", "01d4", "00e4"]
binary_strings = hexa_to_binary(hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(hex_strings, binary_strings):
    print(f"{binary_string}")

0000011010011100
0000010001101100
0000010001011100
0000010000110100
0000001011111100
0000001010100100
0000001001101100
0000000111010100
0000000011100100


In [61]:
# 새로운 입력값
hex_strings = ["045c", "0404", "03b4", "039c", "0374", "034c", "031c", "027c", "022c", "01bc"]
binary_strings = hexa_to_binary(hex_strings)

# 변환된 결과 출력
for hex_string, binary_string in zip(hex_strings, binary_strings):
    print(f"{binary_string}")


0000010001011100
0000010000000100
0000001110110100
0000001110011100
0000001101110100
0000001101001100
0000001100011100
0000001001111100
0000001000101100
0000000110111100


In [62]:
def calculate_error_rates(actual_values, predicted_values):
    """
    여러 개의 실제 값과 예측 값에 대한 오차율을 계산하는 함수.

    매개변수:
    actual_values (list of float): 실제 값 리스트
    predicted_values (list of float): 예측 값 리스트

    반환값:
    list of float: 각 데이터에 대한 오차율 (백분율) 리스트
    """
    if len(actual_values) != len(predicted_values):
        raise ValueError("실제 값 리스트와 예측 값 리스트의 길이는 같아야 합니다.")

    error_rates = []
    for actual, predicted in zip(actual_values, predicted_values):
        if actual == 0:
            error_rate = 0
        else:
            error = actual - predicted
            error_rate = abs((error / actual) * 100)
        error_rates.append(error_rate)

    return error_rates

def calculate_average_error_rate(actual_values, predicted_values):
    """
    여러 개의 실제 값과 예측 값에 대한 평균 오차율을 계산하는 함수.

    매개변수:
    actual_values (list of float): 실제 값 리스트
    predicted_values (list of float): 예측 값 리스트

    반환값:
    float: 평균 오차율 (백분율)
    """
    error_rates = calculate_error_rates(actual_values, predicted_values)
    average_error_rate = sum(error_rates) / len(error_rates)
    return average_error_rate

In [63]:
# 예제 사용
actual_values = [3.447215, 2.4453104, 1.2259121, 1.2043934, 1.1346276, 0.6330924, 0.4353165, -0.2769869, -0.47311664, -0.6537097]
predicted_values = [3.447215, 2.4453104, 1.2259121, 1.2043934, 1.1346276, 0.6330924, 0.4353165, -0.2769869, -0.47311664, -0.6537097]
average_error_rate = calculate_average_error_rate(actual_values, predicted_values)

print(f"최종 오차율: {average_error_rate}%")

최종 오차율: 0.0%


In [64]:
import time
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# 10개의 값을 생성
values = np.random.rand(10)

# 실행 시간 측정
start_time = time.time()
softmax_values = softmax(values)
end_time = time.time()

print(f"Softmax output: {softmax_values}")
print(f"Execution time: {end_time - start_time} seconds")

Softmax output: [0.08277174 0.14726901 0.11833748 0.10356559 0.06652416 0.06652256
 0.06031798 0.13533088 0.10382031 0.11554028]
Execution time: 0.0005555152893066406 seconds


In [65]:
import timeit
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# 10개의 값을 생성
values = np.random.rand(10)

# 실행 시간 측정 (timeit 모듈 사용)
execution_time = timeit.timeit(lambda: softmax(values), number=100000)
print(f"Average execution time over 1000 runs: {execution_time / 100000} seconds")


Average execution time over 1000 runs: 9.411487509996732e-06 seconds


In [66]:
import time

def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

start_time = time.time()
my_function()
end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 2.0017855167388916 seconds


In [67]:
import timeit

def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

execution_time = timeit.timeit(my_function, number=1)
print(f"Execution time: {execution_time} seconds")


Execution time: 2.0014164759995765 seconds


In [68]:
import time

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time} seconds")
        return result
    return wrapper

@measure_time
def my_function():
    # 실행할 코드
    time.sleep(2)  # 예시로 2초 지연

my_function()

Execution time: 2.002105712890625 seconds


In [70]:

import socket

# HW -> SW connect
# This should be fixed later when connecting with hardware.
def receive_from_hardware(host: str, port: int, buffer_size: int = 1024) -> bytes:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        attention_probs_hw = s.recv(buffer_size)
    return attention_probs_hw

# HW 2진수 -> SW Decode
def decode_values(encoded_values_list: list) -> list:
    encoded_values = tuple(encoded_values_list)
    decoded_values = []

    for encoded_value in encoded_values:
        special_value_bit = encoded_value[0]
        if special_value_bit == '1':
            decoded_values.append(0.0)
        else:
            sign_bit = encoded_value[1]
            sign = -1 if sign_bit == '1' else 1

            integer_part = int(encoded_value[2:5], 2)
            fractional_part = int(encoded_value[5:], 2) / (1 << 13)

            decoded_value = sign * (integer_part + fractional_part)
            decoded_values.append(round(decoded_value, 7))

    return decoded_values

# 1D -> 4D 변환 코드
def convert_to_4d(input_list):
    # Step 1: Divide the list into sublists of 10 elements each
    sublists = [input_list[i:i + 10] for i in range(0, len(input_list), 10)]

    # Step 2: Group every 10 sublists into a larger list to form a [12, 10, 10] shape
    grouped_sublists = [sublists[i:i + 10] for i in range(0, len(sublists), 10)]

    # Step 3: Convert the final list to a tensor and add an extra dimension to form [1, 12, 10, 10]
    attention_probs_4d = torch.tensor(grouped_sublists).unsqueeze(0)
    return attention_probs_4d

In [79]:
%mkdir ./model/attention_scores.pkl

In [96]:
import pickle
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers.models.bert.modeling_bert import BertSelfAttention


# 가정: attention_scores는 모델에서 계산된 텐서입니다.
# 예를 들어, 모델의 어떤 forward 메서드에서 attention_scores를 얻었다고 가정합니다.
# 이제 이 값을 파일로 저장하려고 합니다.

# 파일 저장 함수를 정의합니다.
def flatten(lst):
    return [item for sublist in lst for item in sublist]

def save_attention_scores(attention_scores, file_path):
    flat_scores = flatten(attention_scores)  # 리스트 평평하게

    tensor_scores = []
    for item in flat_scores:
        if isinstance(item, dict):
            # 문자열이 아닌 값만 선택
            numeric_values = [value for value in item.values() if isinstance(value, (int, float))]
            tensor_scores.append(torch.tensor(numeric_values))
        else:
            # 텐서로 변환 가능한 값만 추가
            if isinstance(item, (list, int, float)):
                tensor_scores.append(torch.tensor(item))

    # 텐서를 스택으로 결합
    stacked_scores = torch.stack(tensor_scores)
    with open(file_path, 'wb') as f:
        pickle.dump(stacked_scores.cpu().numpy(), f)


save_attention_scores(result, '/./content/model/attention_scores.pkl')